In [1]:
pip install pyspark pyarrow

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 63.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=a6da54d72f2fa656434d65593989d27e715d5d5eeffab650dea6c60c7f0b3a92
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [107]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_3")\
    .config("spark.executor.instances", 2)\
    .config("spark.executor.memory", '2g')\
    .config("spark.executor.cores", 1)\
    .getOrCreate()
sc = spark.sparkContext

In [108]:
from pyspark.sql import Window
import pyspark.sql.functions as F

In [109]:
spark

In [110]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [111]:
loc_data = './gdrive/MyDrive/Colab Notebooks/'

In [112]:
# Reading CSV
df = spark.read.csv(loc_data + 'raw_sales.csv', header=True, inferSchema=True)

In [145]:
df.show()

+-------------------+--------+-------+------------+--------+
|           datesold|postcode|  price|propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|
|2007-06-27 00:00:00|    2906| 305000|       house|       3|
|2007-06-27 00:00:00|    2612| 850000|       house|       4|
|2007-06-28 00:00:00|   

Задание 1
Добавьте к таблице следующие поля:

Средняя стомость 10 проданных домов до текущего в том же районе (4digit 
postcode) (1 балл)

Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)

Стоимость последнего проданного дома до текущего (1 балл)

In [146]:
# AVG Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)

windSpec = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow - 10, Window.currentRow - 1)


df11 = df.withColumn('AVG last 10 to Current', F.avg('price').over(windSpec))
df11.show()

+-------------------+--------+-------+------------+--------+----------------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|
+-------------------+--------+-------+------------+--------+----------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|              564250.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|              509900.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|              479750.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|              520500.0|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|              57

In [147]:
# AVG Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)

windSpec = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow + 1, Window.currentRow + 10)


df12 = df11.withColumn('AVG 10 After Current', F.avg('price').over(windSpec))
df12.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|            742850.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|              564250.0|            786600.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|              509900.0|            839200.0|
|2008-06-19 00:00:00|    2600| 765000|       house|    

In [148]:
# AVG Стоимость последнего проданного дома до текущего (1 балл)

windSpec = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)


df13 = df12.withColumn('Last Price to Current', F.last('price').over(windSpec))
df13.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|Last Price to Current|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|                 null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|               327000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|               790000|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|            742850.0|               825000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|              564250.0|            786600.0|       

Задание 2

Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)

(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)

In [149]:
# Make a first DF 

df2 = df

df2 = df2.withColumn('year', F.year('datesold'))

df2 = df2.select(F.col('year'), F.col('price'))

df2 = df2.groupby('year').avg('price').sort(F.col('year').asc())

df2.show()



+----+-----------------+
|year|       avg(price)|
+----+-----------------+
|2007|522377.2108843537|
|2008|493814.1627543036|
|2009|496092.0329593268|
|2010|559564.8102893891|
|2011|566715.1102265768|
|2012|552501.3692142088|
|2013|553416.2968381312|
|2014|  592653.76143905|
|2015|626101.3426535088|
|2016|635185.3116683726|
|2017| 671880.551420392|
|2018|660701.0427682737|
|2019|634184.2187725632|
+----+-----------------+



In [150]:
# Make a second DF 

df13 = df13.withColumn('year', F.year('datesold'))
df13.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|Last Price to Current|year|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|                 null|2007|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|               327000|2007|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|               790000|2007|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|            742850.0|               825000|2008|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|              56

In [151]:
# Join Dataframes

df4 = df13.join(df2, df13.year == df2.year, 'Inner')
df4.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|Last Price to Current|year|year|       avg(price)|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|                 null|2007|2007|522377.2108843537|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|               327000|2007|2007|522377.2108843537|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|               790000|2007|2007|522377.2108843537|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     64733

Задание 3
В итоге у вас таблица с колонками (или нечто похожее):

price

Среднегодовая цена

Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)

Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)

Стоимость последнего проданного дома до текущего ((1 балл)
и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [152]:
# Distinct window functions are not supported !!!

df4.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|Last Price to Current|year|year|       avg(price)|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|                 null|2007|2007|522377.2108843537|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|               327000|2007|2007|522377.2108843537|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|               790000|2007|2007|522377.2108843537|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     64733

In [161]:
# See All Count Rows

from pyspark.sql.functions import *

df5 = df4.select(count('*').alias('all count rows'))
df5.show()

+--------------+
|all count rows|
+--------------+
|         29580|
+--------------+



In [165]:
# See Unique Count Rows

df5 = df4.select(countDistinct('*').alias('Unique count rows'))

df5.show()

+-----------------+
|Unique count rows|
+-----------------+
|            29526|
+-----------------+



In [167]:
# Adding column With Unique count Rows IN EACH ROW

df4.join(df5).show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|AVG last 10 to Current|AVG 10 After Current|Last Price to Current|year|year|       avg(price)|Unique count rows|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+---------------------+----+----+-----------------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            708350.0|                 null|2007|2007|522377.2108843537|            29526|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|            698350.0|               327000|2007|2007|522377.2108843537|            29526|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|            679350.0|               790000|2007|2007